In [1]:
import cvxpy as cp

# 저자

- TA: 성준모 (Joonmo Sung)
- SYSTEMS MODELING AND PROGRAMMING LAB DEPARTMENT OF INDUSTRIAL ENGINEERING, YONSEI UNIVERSITY - SYMPLY
- 문의: `sjm21314@naver.com`,`sjm21314@yonsei.ac.kr`
***
본 강의자료는 Keha,A. B.,Khowala,K.,&Fowler,J. W. (2009). Mixed integer programming formulations for single machine scheduling problems. Computers & Industrial Engineering, 56(1), 357-367 논문을 바탕으로 제작되었으며,

반도체데이터사이언스 협동과정 수업 용도 이외의 목적으로 저자의 허락 없이 다른 사람들과 공유할 수 없습니다.

## Dataset
### Set
- N = $\{1,2,...,n\}, j \in N$
### Parameters
- $p_j$: processing time of job $j$
- $d_j$: due date of job $j$
- $w_j$: weight of job $j$
- $r_j$: release date of job $j$

In [2]:
# Offline 상황을 가정.

n = 5 # 작업의 개수

N = [_ for _ in range(n)] # 작업 set을 리스트 자료구조를 이용해 0, 1, 2,..., n-1 까지 담아줌
p = [4, 3, 7, 2, 2] # 가공시간 데이터를 리스트 자료구조를 이용해 만들어 줌
d = [5, 6, 8, 8, 17] # 납기 데이터를 리스트 자료구조를 이용해 만들어 줌
w = [5, 4, 3, 2, 1] # 가중치 데이터를 리스트 자료구조를 이용해 만들어 줌
r = [1, 2, 3, 3, 5] # 작업의 release date(도착 시점) 데이터를 리스트 자료구조를 이용해 만들어 줌

In [3]:
obj_select = 'twt' # 'wct', 'tj', 'twt', 'L_max' 중 하나 선택
release_time = False

## 2. Time index variables
### Sets
- T: planning horizon ids discretized into the periods 1,2,3,...T

In [4]:
T = [_ for _ in range((sum(p) + sum(r)) * 2)] # 넉넉하게 (processing time 합 + release time 합)* 2 로 선택T가 작을수록 공간복잡도 감소

### Decision Variables
- $x_{j,t}$: 작업 $j$가 period t에서 시작하면 1, 아니면 0인 이진변수, $x_{j,t} \in \{0, 1\} \ \forall j \in N, \forall t \in T$

In [5]:
# Decision Variable 만들기
x = {(j, t): cp.Variable(boolean = True) for j in N for t in T}  # 이진 변수

### 기본 constraint
- $\sum_{t=1}^{|T|-p_j+1} x_{j,t} = 1 \quad \forall j \in N$
- $\sum_{j=1}^n \sum_{s = max(0, t-p_j+1)}^t x_{j,s} \quad \forall t \in T$

In [10]:
# 제약식 리스트 선언
constraints = []

# 제약식 추가
constraints += [sum(x[j,t] for t in range(0, len(T)-p[j]+1)) == 1 for j in N] # range(0, 5) 면 0, 1, 2, 3, 4 그래서 len(T) 쓴 것임
constraints += [sum(sum(x[j,s] for s in range(max(0, t-p[j]+1), t + 1)) for j in N) <= 1 for t in T] # t+1

#### Release time 반영 제약식
$ x_{j,t} = 0 \quad \forall t \leq r_j, \forall j \in J $

In [11]:
if release_time == True:
    constraints += [x[j,t] == 0 for j in N for t in T if t + 1 <= r[j]]

#### To minimize the total weighted completion time
$ Minimize \sum_{j = 1}^n\sum_{t=1}^{|T|-p_j+1} \xi_{j,t} x_{j,t}$

$ \\ \xi_{j,t} = w_j(t-1+p_j) \quad \forall j \in N, \forall t \in T$

In [12]:
if obj_select == 'wct':
    # 크사이 데이터 셋을 리스트로 구현하여 사용
    xi = [[w[j]*(t + p[j]) for t in T] for j in N] # t = 0 부터 시작하기에 1을 빼줄 필요 없음 논문은 1부터 시작해서 그런 것임

    # 목적식
    obj = cp.Minimize(sum(sum(xi[j][t] * x[j,t] for t in range(0, len(T)-p[j]+1)) for j in N))

#### To minimize the number of tardy jobs
$ Minimize \sum_{j = 1}^n\sum_{t=1}^{|T|-p_j+1} \xi_{j,t} x_{j,t}$

$ \\ \xi_{j,t} = 1, \  if \  t > (d_j-p_j+1), \quad \forall j \in N, \forall t \in T \ 0, \ otherwise $

In [14]:
if obj_select == 'tj':
    # 크사이 데이터 셋을 리스트로 구현하여 사용
    xi = [[1 if t > (d[j] - p[j]) else 0 for t in T] for j in N] # t = 0 부터 시작하기에 1을 빼줄 필요 없음 논문은 1부터 시작해서 그런 것임

    # 목적식
    obj = cp.Minimize(sum(sum(xi[j][t] * x[j,t] for t in range(0, len(T)-p[j]+1)) for j in N))

#### To minimize the total weighted tardiness
$ Minimize \sum_{j = 1}^n\sum_{t=1}^{|T|-p_j+1} \xi_{j,t} x_{j,t}$

$ \\ \xi_{j,t} = w_j\max(0, t - 1 + p_j - d_j), \quad \forall j \in N, \forall t \in T$

In [15]:
if obj_select == "twt":
    # 크사이 데이터 셋을 리스트로 구현하여 사용
    xi = [[w[j]*max(0, t + p[j] - d[j]) for t in T] for j in N] # t = 0 부터 시작하기에 1을 빼줄 필요 없음 논문은 1부터 시작해서 그런 것임

    # 목적식
    obj = cp.Minimize(sum(sum(xi[j][t] * x[j,t] for t in range(0, len(T)-p[j]+1)) for j in N))

#### To minimize $L_{max}$
$ Minimize \ L_{max}$

$ \\ s.t. \  L_{max} \geq (C_j - d_j) \quad \forall j \in N$

$ \\ C_j = \sum_{t=1}^{|T|-p_j+1}(t-1+p_j)x_{j,t} \quad \forall j \in N$

In [16]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = cp.Variable(nonneg = True)

    # Lmax 제약식
    constraints += [L_max >= (sum((t+p[j]) * x[j,t] for t in range(0, len(T)-p[j]+1))- d[j]) for j in N] # C_j 대신 바로 x제약식 넣어줌. 그리고 t = 0부터 시작하니까 t-1 해줄 필요 없음

    # 목적식
    obj = cp.Minimize(L_max)

In [17]:
prob = cp.Problem(obj, constraints)
prob.solve()

np.float64(31.0)

In [18]:
print("최적해 상태:", prob.status)
print("최적값:", prob.value)

최적해 상태: optimal
최적값: 31.0


In [19]:
for (j, t), dv in x.items():
    if dv.value > 1e-6:
        print(f"x_{j+1}_{t}: {dv.value}")

x_1_0: 1.0
x_2_4: 1.0
x_3_9: 1.0
x_4_7: 1.0
x_5_16: 1.0
